In [1]:
!pip install google-generativeai requests beautifulsoup4 pandas

In [ ]:
import google.generativeai as genai
import pandas as pd
from datetime import datetime
import re
import random
from typing import Dict, List, Optional
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

class CompetitorAnalysisConfig:
    """Configuration settings for the competitor analysis system"""

    def __init__(self):
        # API Keys - Replace with your actual keys
        self.GEMINI_API_KEY = "ENTER_YOUR_GEMINI_API_KEY"
        self.GOOGLE_PLACES_API_KEY = "ENTER_YOUR_PLACES_API_KEY"

        # Model settings
        self.MODEL_NAME = "models/gemini-1.5-flash"

        # Search parameters
        self.SEARCH_RADIUS = 2000
        self.MAX_COMPETITORS = 10

class CompetitorSearchTool:
    """Tool for finding and analyzing competitor information"""

    def __init__(self, config: CompetitorAnalysisConfig):
        self.config = config

    def search_competitors(self, location: str) -> List[Dict]:
        """Search for competitors in the specified location"""
        # Mock data - Replace with actual Google Places API integration
        mock_competitors = [
            {
                "name": "Fashion Central",
                "address": f"Near {location}",
                "type": "Contemporary Fashion",
                "distance": 0.3,
                "rating": 4.1,
                "price_level": 2
            },
            {
                "name": "Trendy Threads",
                "address": f"Central {location}",
                "type": "Youth Fashion",
                "distance": 0.5,
                "rating": 4.3,
                "price_level": 3
            },
            {
                "name": "Style Studio",
                "address": f"Main Street, {location}",
                "type": "Designer Wear",
                "distance": 0.8,
                "rating": 4.6,
                "price_level": 4
            }
        ]
        return mock_competitors

    def get_footfall_data(self, competitor_name: str) -> Dict:
        """Get footfall data for a specific competitor"""
        current_hour = datetime.now().hour

        # Base footfall calculation
        base_footfall = random.randint(40, 90)

        # Adjust for peak hours
        if 11 <= current_hour <= 13 or 18 <= current_hour <= 20:
            base_footfall += random.randint(10, 25)
        elif current_hour < 10 or current_hour > 21:
            base_footfall -= random.randint(20, 30)

        base_footfall = max(10, min(100, base_footfall))

        return {
            "current_footfall": base_footfall,
            "avg_daily_visitors": random.randint(200, 500),
            "peak_hours": ["11:00-13:00", "18:00-20:00"],
            "footfall_trend": "increasing" if base_footfall > 60 else "stable"
        }

class CompetitorAnalysisPipeline:
    """Main pipeline for competitor analysis using conversational AI"""

    def __init__(self, config: CompetitorAnalysisConfig):
        self.config = config
        self.search_tool = CompetitorSearchTool(config)

        # Initialize Gemini AI
        genai.configure(api_key=config.GEMINI_API_KEY)
        self.model = genai.GenerativeModel(model_name=config.MODEL_NAME)

    def analyze_competitors(self, location: str, user_query: str = "") -> Dict:
        """Main function that searches for competitors and generates analysis"""
        logging.info(f"Searching for competitors in {location}")

        # Get competitor data
        competitors = self.search_tool.search_competitors(location)

        # Enhance with footfall data
        detailed_competitors = []
        for comp in competitors:
            footfall_data = self.search_tool.get_footfall_data(comp["name"])
            comp.update(footfall_data)
            detailed_competitors.append(comp)

        # Generate AI analysis
        analysis = self._generate_ai_analysis(location, detailed_competitors, user_query)

        return {
            "location": location,
            "competitors": detailed_competitors,
            "analysis": analysis,
            "timestamp": datetime.now().isoformat(),
            "total_competitors": len(detailed_competitors)
        }

    def _generate_ai_analysis(self, location: str, competitors: List[Dict], user_query: str) -> str:
        """Generate comprehensive AI analysis using Gemini"""
        competitor_context = self._format_competitor_data(competitors)

        prompt = f"""
        You are an expert retail market analyst specializing in clothing store competition analysis.

        LOCATION: {location}
        CURRENT TIME: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

        COMPETITOR DATA:
        {competitor_context}

        USER QUERY: {user_query if user_query else "Provide comprehensive competitor analysis"}

        Please provide a detailed analysis covering:
        1. MARKET OVERVIEW - Market saturation and competitive intensity
        2. FOOTFALL ANALYSIS - Current trends and peak hours
        3. COMPETITIVE POSITIONING - Market gaps and opportunities
        4. STRATEGIC RECOMMENDATIONS - Immediate and long-term actions
        5. TACTICAL INSIGHTS - Operating hours and pricing strategies

        Format your response professionally with specific actionable insights.
        """

        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            logging.error(f"Error generating analysis: {str(e)}")
            return f"Error generating analysis: {str(e)}"

    def _format_competitor_data(self, competitors: List[Dict]) -> str:
        """Format competitor data for AI analysis"""
        formatted_data = []

        for i, comp in enumerate(competitors, 1):
            formatted_data.append(f"""
            COMPETITOR {i}: {comp['name']}
            - Type: {comp['type']}
            - Distance: {comp['distance']}km
            - Rating: {comp['rating']}/5.0
            - Price Level: {'$' * comp['price_level']}
            - Current Footfall: {comp['current_footfall']}%
            - Daily Visitors: {comp['avg_daily_visitors']}
            - Peak Hours: {', '.join(comp['peak_hours'])}
            - Trend: {comp['footfall_trend']}
            """)

        return '\n'.join(formatted_data)

    def _extract_location(self, text: str) -> Optional[str]:
        """Extract location from user input"""
        location_patterns = [
            r'in\s+([A-Za-z\s,]+)',
            r'at\s+([A-Za-z\s,]+)',
            r'near\s+([A-Za-z\s,]+)',
            r'around\s+([A-Za-z\s,]+)'
        ]

        for pattern in location_patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                return match.group(1).strip()
        return None

    def display_results(self, result: Dict):
        """Display analysis results in a formatted manner"""
        print("\n" + "="*80)
        print(f"📊 COMPETITOR ANALYSIS REPORT - {result['location'].upper()}")
        print("="*80)

        print(f"\n📈 SUMMARY:")
        print(f"• Total Competitors Found: {result['total_competitors']}")
        print(f"• Analysis Timestamp: {result['timestamp']}")

        print(f"\n🏪 COMPETITORS OVERVIEW:")
        for i, comp in enumerate(result['competitors'], 1):
            status = ("🔥 HIGH TRAFFIC" if comp['current_footfall'] > 70
                     else "📊 MODERATE" if comp['current_footfall'] > 40
                     else "📉 LOW")
            print(f"{i}. {comp['name']} ({comp['type']}) - {status}")
            print(f"   Distance: {comp['distance']}km | Rating: {comp['rating']}/5 | Footfall: {comp['current_footfall']}%")

        print(f"\n🤖 AI ANALYSIS:")
        print("-" * 50)
        print(result['analysis'])
        print("-" * 50)

    def run_interactive_mode(self):
        """Interactive conversational interface"""
        print("🏪 Welcome to the Clothing Store Competitor Analysis System!")
        print("=" * 60)

        while True:
            print("\n🤖 How can I help you with competitor analysis today?")
            print("Examples:")
            print("- 'Analyze competitors in Koramangala, Bangalore'")
            print("- 'Find peak hours for nearby stores'")
            print("- Type 'quit' to exit")

            user_input = input("\n💬 Your query: ").strip()

            if user_input.lower() in ['quit', 'exit', 'bye']:
                print("👋 Thanks for using the Competitor Analysis System!")
                break

            # Extract location
            location = self._extract_location(user_input)
            if not location:
                location = input("📍 Please specify your location: ")

            # Process request
            print(f"\n🔄 Processing your request for {location}...")
            try:
                result = self.analyze_competitors(location, user_input)
                self.display_results(result)

                # Follow-up question
                follow_up = input("\n❓ Any follow-up questions? (press Enter to continue): ")
                if follow_up.strip():
                    print(f"\n🤖 Follow-up Analysis:")
                    follow_up_analysis = self._generate_ai_analysis(location, result['competitors'], follow_up)
                    print(follow_up_analysis)

            except Exception as e:
                logging.error(f"Analysis failed: {str(e)}")
                print(f"❌ Analysis failed: {str(e)}")

class ReportGenerator:
    """Generate various types of reports"""

    @staticmethod
    def generate_csv_report(result: Dict) -> str:
        """Generate CSV report of competitors"""
        df = pd.DataFrame(result['competitors'])
        filename = f"competitor_analysis_{result['location'].replace(' ', '_')}_{datetime.now().strftime('%Y%m%d_%H%M')}.csv"
        df.to_csv(filename, index=False)
        return filename

    @staticmethod
    def generate_text_report(result: Dict) -> str:
        """Generate text report"""
        report = f"""
CLOTHING STORE COMPETITOR ANALYSIS REPORT
Generated: {result['timestamp']}
Location: {result['location']}

{result['analysis']}

DETAILED COMPETITOR DATA:
{chr(10).join([f"{comp['name']} - {comp['type']} - {comp['distance']}km - {comp['rating']}/5"
               for comp in result['competitors']])}
        """
        return report

def quick_analysis(location: str):
    """Quick analysis function for immediate results"""
    config = CompetitorAnalysisConfig()
    pipeline = CompetitorAnalysisPipeline(config)

    print(f"🔍 Analyzing competitors in {location}...")
    result = pipeline.analyze_competitors(location)
    pipeline.display_results(result)

    return result

def main():
    """Main execution function"""
    print("🚀 Clothing Store Competitor Analysis Pipeline")
    print("Choose your preferred mode:")
    print("1. Interactive Conversational Mode")
    print("2. Quick Analysis Mode")

    choice = input("Enter your choice (1 or 2): ")

    config = CompetitorAnalysisConfig()
    pipeline = CompetitorAnalysisPipeline(config)

    if choice == "1":
        pipeline.run_interactive_mode()
    elif choice == "2":
        location = input("📍 Enter your location: ")
        quick_analysis(location)
    else:
        print("Invalid choice. Starting conversational mode...")
        pipeline.run_interactive_mode()

if __name__ == "__main__":
    main()

🚀 Clothing Store Competitor Analysis Pipeline
Choose your preferred mode:
1. Interactive Conversational Mode
2. Quick Analysis Mode
Enter your choice (1 or 2): 1
🏪 Welcome to the Clothing Store Competitor Analysis System!

🤖 How can I help you with competitor analysis today?
Examples:
- 'Analyze competitors in Koramangala, Bangalore'
- 'Find peak hours for nearby stores'
- Type 'quit' to exit

💬 Your query: Analyze competitors in Koramangala, Bangalore

🔄 Processing your request for Koramangala, Bangalore...

📊 COMPETITOR ANALYSIS REPORT - KORAMANGALA, BANGALORE

📈 SUMMARY:
• Total Competitors Found: 3
• Analysis Timestamp: 2025-06-18T02:48:58.642091

🏪 COMPETITORS OVERVIEW:
1. Fashion Central (Contemporary Fashion) - 📉 LOW
   Distance: 0.3km | Rating: 4.1/5 | Footfall: 21%
2. Trendy Threads (Youth Fashion) - 📉 LOW
   Distance: 0.5km | Rating: 4.3/5 | Footfall: 40%
3. Style Studio (Designer Wear) - 📉 LOW
   Distance: 0.8km | Rating: 4.6/5 | Footfall: 24%

🤖 AI ANALYSIS:
---------------